In [1]:
import torch
import torchvision
import numpy as np

## Numpy

In [2]:
##Initialize tensor
a = torch.ones(5)
print(a)


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]



In [3]:
#Convert tensor to numpy array
b = a.numpy()
print(b)

[ 1.  1.  1.  1.  1.]


In [4]:
#Convert numpy array to tensor
a = np.ones(5)
b = torch.from_numpy(a)
print(b)


 1
 1
 1
 1
 1
[torch.DoubleTensor of size 5]



In [ ]:
# let us run this cell only if CUDA is available
x = torch.ones(5)
y = torch.ones(5)
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y
    
def add():
    x + y
%timeit -n 1 add()

A microsecond µs is 1 millionth of a second

### Tensor

http://pytorch.org/docs/tensors.html

In [ ]:
#2D Tensor
x = torch.FloatTensor(
    [[1, 2, 3], 
     [4, 5, 6]]
)
print(x.size())
print(x)

In [ ]:
#3D Tensor
x = torch.FloatTensor([
    [[1, 1, 1], 
     [2, 2, 2]],
    [[3, 3, 3], 
     [4, 4, 4]]
])
print(x.size())
print(x)

In [ ]:
x = torch.randn(2,2,3)
x

## Autograd

In [6]:
from torch.autograd import Variable
from torch import Tensor

In [7]:
x = Variable(torch.ones(2,2), requires_grad=True)
print (x)
print (x.data)
print (x.creator)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]


 1  1
 1  1
[torch.FloatTensor of size 2x2]

None


In [8]:
y = x + 2
print (y)
print (y.creator)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [9]:
z = y*y*3
out = z.mean()
out

Variable containing:
 27
[torch.FloatTensor of size 1]

In [10]:
out.backward()

In [11]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



## Neural Networks

In [12]:
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120) # an affine operation: y = Wx + b
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2)) #2x2 pool window
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) #if shape is square, you only need to define one number
        x = x.view(-1, self.num_flat_features(x)) #Flatten()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    ## Backward() function is automatically defined for you!!
    
    def num_flat_features(self, x):
        #basically counting parameters and flattening 
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [14]:
net = NeuralNetwork()
print (net)

NeuralNetwork (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [15]:
X = Variable(torch.randn(1,1,32,32)) #nSamples x nChannels x Height x Width
yHat = net(X)
yHat

Variable containing:
-0.1277 -0.1156 -0.0090  0.0170 -0.0432  0.0437 -0.0380  0.0414 -0.0064 -0.0183
[torch.FloatTensor of size 1x10]

## Loss Function

http://pytorch.org/docs/nn.html#loss-functions

In [16]:
yHat = net(X)
target = Variable(torch.range(1, 10))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(yHat, target)
print(loss)

Variable containing:
 38.5931
[torch.FloatTensor of size 1]



In [17]:
print(loss.creator)  # MSELoss
print(loss.creator.previous_functions[0][0])  # Linear
print(loss.creator.previous_functions[0][0].previous_functions[0][0])  # ReLU

## Backprop

In [18]:
#First clear the existing gradients!! 
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

#Backprop as simple as..
loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
 -8.7405
 -6.8205
  5.9301
 -5.5069
 -4.7259
 -5.9578
[torch.FloatTensor of size 6]



## Update Weights

In [19]:
#SGD
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)  #subtract gradient from weights...

In [20]:
#Other optimizers
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [21]:
#using the optimizers
#Zero the gradient
optimizer.zero_grad()
yHat = net(X)
loss = criterion(yHat,target)
loss.backward()
optimizer.step() #update the weights

## CNNs

https://chsasank.github.io/pytorch-tutorials/beginner/blitz/cifar10_tutorial.html

### Concat

http://pytorch.org/docs/torch.html#torch.cat

In [53]:
#Does this work?
torch.manual_seed??
torch.cuda.manual_seed(1)

In [103]:
x = torch.randn(2,3)
x


-0.2067  1.0672  0.1732
-0.6873  0.3111  0.2358
[torch.FloatTensor of size 2x3]

In [104]:
# 2D - combine new tensors as new rows stacked on stop of each other
torch.cat([x,x,x], 0) #0 = row = first dimension of input


-0.2067  1.0672  0.1732
-0.6873  0.3111  0.2358
-0.2067  1.0672  0.1732
-0.6873  0.3111  0.2358
-0.2067  1.0672  0.1732
-0.6873  0.3111  0.2358
[torch.FloatTensor of size 6x3]

In [105]:
# 2D - combine new tensors as new columns stacked next to each other
torch.cat([x,x,x], 1) #1 = column = 2nd dimension of input


-0.2067  1.0672  0.1732 -0.2067  1.0672  0.1732 -0.2067  1.0672  0.1732
-0.6873  0.3111  0.2358 -0.6873  0.3111  0.2358 -0.6873  0.3111  0.2358
[torch.FloatTensor of size 2x9]

In [106]:
# 2D - Stack - combine tensors along new dimension - 3D now
torch.stack([x,x,x],0)


(0 ,.,.) = 
 -0.2067  1.0672  0.1732
 -0.6873  0.3111  0.2358

(1 ,.,.) = 
 -0.2067  1.0672  0.1732
 -0.6873  0.3111  0.2358

(2 ,.,.) = 
 -0.2067  1.0672  0.1732
 -0.6873  0.3111  0.2358
[torch.FloatTensor of size 3x2x3]

In [108]:
#3D Tensor
x = torch.FloatTensor([
    [[1, 1, 1], 
     [2, 2, 2]],
    [[3, 3, 3], 
     [4, 4, 4]]
])
print(x.size())
print(x)

torch.Size([2, 2, 3])

(0 ,.,.) = 
  1  1  1
  2  2  2

(1 ,.,.) = 
  3  3  3
  4  4  4
[torch.FloatTensor of size 2x2x3]



In [113]:
out = torch.cat([x,x],0)
print(out.size())
print(out)

torch.Size([4, 2, 3])

(0 ,.,.) = 
  1  1  1
  2  2  2

(1 ,.,.) = 
  3  3  3
  4  4  4

(2 ,.,.) = 
  1  1  1
  2  2  2

(3 ,.,.) = 
  3  3  3
  4  4  4
[torch.FloatTensor of size 4x2x3]



In [115]:
out = torch.cat([x,x],1)
print(x.size())
print(out.size())
print(out)

torch.Size([2, 2, 3])
torch.Size([2, 4, 3])

(0 ,.,.) = 
  1  1  1
  2  2  2
  1  1  1
  2  2  2

(1 ,.,.) = 
  3  3  3
  4  4  4
  3  3  3
  4  4  4
[torch.FloatTensor of size 2x4x3]



## Conv Transpose

* http://deeplearning.net/software/theano/tutorial/conv_arithmetic.html
* no padding = o' = i' + (k - 1)
* w padding = o' = i' + (k - 1) - 2p
* w stride = o' = s (i' - 1) + k

In [ ]:
layer = nn.ConvTranspose2d(in_channels=80, 
       out_channels=80, kernel_size=3, stride=2, padding=0, bias=True)
input = torch.randn(1, 80, 11, 15)
layer(Variable(input)).size()

In [ ]:
layer = nn.ConvTranspose2d(in_channels=80, 
       out_channels=80, kernel_size=3, stride=2, padding=0, bias=True) #outputdim = s(i-1) + k
input = torch.randn(1, 80, 11, 15)
k = input.dim() - 2 #get num dimensions and subtract 2
min_sizes = [dim_size(layer, input, d) for d in range(k)] #This just gets the dimension sizes of output after transpose
#So it can't go any smaller, only adds padding - minimum output size is the output of the normal transpose
#min_size = output of normal transpose
#max_size = 
[min_sizes[d] + layer.stride[d] - 1 for d in range(k)]

## Center Crop

In [ ]:
#https://github.com/pytorch/pytorch/blob/master/torch/nn/_functions/conv.py#L59
#https://github.com/Lasagne/Lasagne/blob/master/lasagne/layers/merge.py#L162
#Author does a center crop which crops both inputs (skip and upsample) to size of minimum dimension on both w/h
#But does this get us back to the output image size then?
def center_crop(layer, max_height, max_width):
    print("maxwidth", max_width)
    print("maxheigth", max_height)

    batch_size, n_channels, layer_height, layer_width = layer.size()
    print('layer', batch_size, n_channels, layer_height, layer_width)
    xy1 = (layer_width - max_width) // 2
    xy2 = (layer_height - max_height) // 2
    print('xy1', xy1)
    print('xy2', xy2)
    return layer[:, :, xy2:(xy2 + max_height), xy1:(xy1 + max_width)]

In [ ]:
t1 = Variable(torch.randn(1,80,11,15))
skip = Variable(torch.randn(1, 448, 22, 30))
layer = nn.ConvTranspose2d(in_channels=80, 
       out_channels=80, kernel_size=3, stride=2, padding=0, bias=True) #outputdim = s(i-1) + k
out = layer(t1)